In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt; plt.ion()
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
# import Planner
import os
from IPython.display import display, clear_output

In [ ]:

def load_map(fname):
    '''
    Loads the boundary and blocks from map file fname.

    boundary = [['xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b']]

    blocks = [['xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b'],
                ...,
                ['xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b']]
    '''
    mapdata = np.loadtxt(fname,dtype={'names': ('type', 'xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b'),\
                                        'formats': ('S8','f', 'f', 'f', 'f', 'f', 'f', 'f','f','f')})
    blockIdx = mapdata['type'] == b'block'
    boundary = mapdata[~blockIdx][['xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b']].view('<f4').reshape(-1,11)[:,2:]
    blocks = mapdata[blockIdx][['xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax','r','g','b']].view('<f4').reshape(-1,11)[:,2:]
    return boundary, blocks

In [ ]:
def draw_block_list(ax,blocks):
    '''
    Subroutine used by draw_map() to display the environment blocks
    '''
    v = np.array([[0,0,0],[1,0,0],[1,1,0],[0,1,0],[0,0,1],[1,0,1],[1,1,1],[0,1,1]],dtype='float')  #Vertices 
    f = np.array([[0,1,5,4],[1,2,6,5],[2,3,7,6],[3,0,4,7],[0,1,2,3],[4,5,6,7]])                    # Faces
    clr = blocks[:,6:]/255                                                                         # color
    n = blocks.shape[0]                                                                            # Number of block
    d = blocks[:,3:6] - blocks[:,:3]                                                               # diagonal vectors of all blocks
    vl = np.zeros((8*n,3))                                                                         # Vertex list
    fl = np.zeros((6*n,4),dtype='int64')                                                           # Face list
    fcl = np.zeros((6*n,3))                                                                        # Face color list
    for k in range(n):                   
        vl[k*8:(k+1)*8,:] = v * d[k] + blocks[k,:3]                                                # Add to the vertices and faces
        fl[k*6:(k+1)*6,:] = f + k*8
        fcl[k*6:(k+1)*6,:] = clr[k,:]

    if type(ax) is Poly3DCollection:
        ax.set_verts(vl[fl])
    else:
        pc = Poly3DCollection(vl[fl], alpha=0.25, linewidths=1, edgecolors='k')
        pc.set_facecolor(fcl)
        h = ax.add_collection3d(pc)
        return h

In [ ]:
def draw_map(boundary, blocks, start = None, goal = None, points=None, nodes=None, fig = None, ax = None, figtitle = None, axtitle = None):
    '''
    Visualization of a planning problem with environment boundary, obstacle blocks, and start and goal points
    '''
    if nodes:
        blocks = np.vstack(blocks, nodes)
    # if fig is None:
    if ax is None and fig is None:
        fig = plt.figure()
        ax = fig.add_subplot(121, projection='3d')
    elif fig is not None:
        ax = fig.add_subplot(122, projection ='3d')
    hb = draw_block_list(ax,blocks)
    hs = ax.plot(start[0:1],start[1:2],start[2:],'co',markersize=7,markeredgecolor='b') if start is not None else None
    hg = ax.plot(goal[0:1],goal[1:2],goal[2:],'go',markersize=7,markeredgecolor='k')  if goal is not None else None
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(boundary[0,0],boundary[0,3])
    ax.set_ylim(boundary[0,1],boundary[0,4])
    ax.set_zlim(boundary[0,2],boundary[0,5])
    if points is not None:
        ax.plot(points[0,:],points[1,:],points[2,:],'r')
    if figtitle is not None:
        fig.suptitle(figtitle)
    if axtitle is not None:
        ax.set_title(axtitle)
    
    return fig, ax, hb, hs, hg

In [ ]:
maps = (os.listdir('./starter_code/maps/'))
start = np.array([2.3, 2.3, 2.3])
goal = np.array([6.0, 6.0, 5.5])
print(maps)

In [ ]:
# for mapname in maps:
#     mapfile = os.path.join('./starter_code/maps/', mapname)
#     boundary, blocks = load_map(mapfile)
#     fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal)
#     print(mapname)
mapfile = os.path.join('./starter_code/maps/', 'single_cube.txt')
boundary, blocks = load_map(mapfile)

# fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points=None)
# boundary, blocks

So basically Ihave to vis z map, viz a start and goal point, viz a line between them (use the cube world)
Then implement a collision checker
Then 

In [ ]:
# def line_segment_collides_here(line, obstacle):
#     t_min = 0.0
#     t_max = 1.0
#     P0 = line[0:4]
#     P1 = line[3:6]
#     d = [P1[i] - P0[i] for i in range(3)]
#     obstacle_min = obstacle[0:4]
#     obstacle_max = obstacle[3:6]
    
#     for i in range(3):  # For x, y, z axes
#         if d[i] != 0.0:
#             t1 = (obstacle_min[i] - P0[i]) / d[i]
#             t2 = (obstacle_max[i] - P0[i]) / d[i]
#             t_min_axis = min(t1, t2)
#             t_max_axis = max(t1, t2)
#             t_min = max(t_min, t_min_axis)
#             t_max = min(t_max, t_max_axis)
#         else:
#             if P0[i] < obstacle_min[i] or P0[i] > obstacle_max[i]:
#                 return False  # The segment is parallel to the slab and outside
    
#     return t_min <= t_max

# def line_segment_collides(line, obstacles):
#     for obstacle in obstacles:
#         if line_segment_collides_here(line, obstacle):
#             return True
#     return False


# line = np.array([0, 2.1, 1, 2.1, 0, 1])
# obstacle = np.array([0, 0, 0, 1, 1, 1])



TOimplement the planner, I need to get the horizon in the form of a possible next states
I need to have a heuristic
And I have to choose a planner to implement
How to implement A-star
Have an open list
Have a pr

In [ ]:
from copy import deepcopy
from itertools import product

G = 0  # Cost index
P = 1  # Parent Index 
H = 2  # Heuristic Index
F = 3  # f Index



class LabelCorrecting(object):

    def __init__(self, boundary, blocks, bbwidth, diagonals_allowed = False, dim = 3):
        self.boundary = boundary[0]
        self.blocks = blocks 
        self.bbwidth = bbwidth 
        self.da = diagonals_allowed
        self.dim = dim
        # self.fig, self.ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points= None)
        # display(fig)
        # clear_output(wait= True)


    class Terminus:
        def __init__(self, outer_class, point, type = ''):
            self.outer_class = outer_class
            self.point = (point)
            self.bb = np.array(self.outer_class.bb_from_point(point))
            if type == 'goal':
                self.bb[2*self.outer_class.dim:] = (218,78,48)
            elif type == 'start':
                self.bb[2*self.outer_class.dim:] = (94, 218, 48)
            self.center = tuple(np.round(0.5*(self.bb[0:self.outer_class.dim] + self.bb[self.outer_class.dim:2*self.outer_class.dim]), 3))


    def bb_from_point(self, point):
        point = np.array(point)
        boundary_min = self.boundary[:self.dim]
        bb_min = point - ((point- boundary_min) % self.bbwidth)
        bb_max = bb_min + self.bbwidth
        return (np.hstack([bb_min, bb_max, np.zeros((3,))]))
    
    def outside_the_boundary(self, traj_box):
        boundary = self.boundary
        for d in range(self.dim):
            dmin = d
            dmax = d + self.dim 
            if np.any(traj_box[dmin] < boundary[dmin]) or np.any(traj_box[dmax] > boundary[dmax]):
                return True
        return False
        
        
    def box_colliding(self,traj_box):
        blocks = self.blocks
        # traj_box = np.array(traj_box)
        xmin = 0
        ymin = 1
        zmin = 2

        xmax = self.dim
        ymax = self.dim + 1
        zmax = self.dim + 2
        

        if np.any(traj_box[xmin] > blocks[:,xmax]) or np.any(traj_box[xmax] < blocks[:,xmin]):
            return False
        if np.any(traj_box[ymin] > blocks[:,ymax]) or np.any(traj_box[ymax] < blocks[:,ymin]):
            return False
        if self.dim ==3:
            if np.any(traj_box[zmin] > blocks[:,zmax]) or np.any(traj_box[zmax] < blocks[:,zmin]):
                return False
        return True
    
    def heuristic(self, point):
        if self.da:
            ## return Euclidean 
            
            return max(0, np.linalg.norm(np.subtract(point, self.goal.center)))
        else :
            ## return Manhattan
            return max(0, np.sum(np.abs(np.subtract(point, self.goal.center))))
            pass
    def cost(self, point_old, point_new):
        return np.linalg.norm(np.subtract(point_new, point_old))
    
    def children(self,node):
        children_list = []
        if self.da:
            adj_list = (list(product([-1.,0.,1.], repeat = self.dim)))
            adj_list.remove(tuple([0.]*self.dim))
            
        else:
            if self.dim == 3:
                adj_list = [(-1.,0.,0.),(1.,0.,0.), (0.,1.,0.), (0.,-1.,0.), (0.,0.,1.), (0.,0.,-1.)]
            elif self.dim == 2:
                adj_list = [(-1.,0.),(1.,0.), (0.,1.), (0.,-1.)]
        
        for adj in adj_list:
            child = np.round(np.add(node, np.multiply(adj, self.bbwidth)), 4)
            child_box = self.bb_from_point(child)
            # print(f"        The adj is {adj}, and the child is {child}, with the child box {child_box}")
            if self.box_colliding(child_box):
                # print(f"        There is a collision with a box {self.blocks}")
                continue
            elif self.outside_the_boundary(child_box):
                # print(f"        There is a collision with a boundary {self.boundary}")
                continue
            else:
                children_list.append(child)


        return children_list
    
    def pop_open(self):
        min_f = np.inf
        node = None
        node_val = None
        for key, value in self.open.items():
            # if value[G] < min_f:
            if value[G] + value[H] < min_f:
                min_f = value[G] + value[H]
                # min_f = value[G]
                node = key
                node_val = value
        assert node is not None
        self.open.pop(node)
        return node, node_val
        
    def plan(self, start, goal):
        self.start = self.Terminus(outer_class = self, point = start, type='start')
        self.goal  = self.Terminus(outer_class = self, point = goal, type='goal')


        self.open = {}
        self.open[self.start.center] = [0, None, self.heuristic(self.start.center),self.heuristic(self.start.center) ] ## G, PARENT, H, F

        self.closed = {}

        if self.box_colliding(self.start.bb) :
            print(f"The start is colliding")
            return None, None
        if self.outside_the_boundary(self.start.bb):
            print(f"The start is outside the boundary")
            return None, None
        if self.box_colliding(self.goal.bb) :
            print(f"The goal is colliding")
            return None, None
        if self.outside_the_boundary(self.goal.bb):
            print(f"The goal is outside the boundary")
            return None, None
            

        while self.goal.center not in self.closed.keys():

            node, node_val = self.pop_open()  ### Visualize


            self.closed[tuple(node)] = node_val
            
            for child in self.children(node):
                # print(f"    We explore a child called {child}")
                if tuple(child) in self.closed.keys():
                    # print(f"    The node's child is {child} which is in closed ")
                    continue
                if tuple(child) in self.open.keys():
                    # print(f"    The node's child is {child} which is in open ")
                    if  self.open[tuple(child)][G] > node_val[G] + self.cost(node, child):
                        self.open[tuple(child)][G] = node_val[G] + self.cost(node, child)
                        self.open[tuple(child)][P] = node
                        self.open[tuple(child)][F] = self.open[tuple(child)][G] + self.open[tuple(child)][H]
                else:
                    # print(f"    The node's child is {child} which is being discovered")
                    # if  self.open[child][G] > node_val[G] + self.cost(node, child):
                    c_ij = self.cost(node, child)
                    g_i  = node_val[G]
                    h_j = g_i + c_ij
                    h_j  = self.heuristic(child)
                    g_j  = g_i + c_ij
                    f_j = g_j + h_j
                    self.open[tuple(child)] = [g_j , node, h_j, f_j]
            # self.fig, self.ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points=None, fig = self.fig, ax = self.ax)
            # display(fig)
            # clear_output(wait= False)

        path = [self.goal.center]
        # total_cost = self.closed[self.goal.center][G]
        cost = 0
        while self.start.center not in path:
            node = path[-1]
            parent = self.closed[node][P]
            path.append(parent) ## Visualize
            cost += self.cost(node, parent)
        path.reverse()
        return path, cost


class A_star(LabelCorrecting):
    def __init__(self, boundary, blocks, bbwidth, diagonals_allowed = False, dim = 3):
        super(A_star, self).__init__(boundary, blocks, bbwidth, diagonals_allowed = False, dim = 3)
    
    def pop_open(self):
        min_f = np.inf
        node = None
        node_val = None
        for key, value in self.open.items():
            # if value[G] < min_f:
            if value[G] + value[H] < min_f:
                min_f = value[G] + value[H]
                # min_f = value[G]
                node = key
                node_val = value
        assert node is not None
        self.open.pop(node)
        return node, node_val
        

class Djikstra(LabelCorrecting):
    def __init__(self, boundary, blocks, bbwidth, diagonals_allowed = False, dim = 3):
        super(Djikstra, self).__init__(boundary, blocks, bbwidth, diagonals_allowed = False, dim = 3)
    
    def pop_open(self):
        min_f = np.inf
        node = None
        node_val = None
        for key, value in self.open.items():
            if value[G] < min_f:
            # if value[G] + value[H] < min_f:
                # min_f = value[G] + value[H]
                min_f = value[G]
                node = key
                node_val = value
        assert node is not None
        self.open.pop(node)
        return node, node_val
        

In [ ]:
planner = A_star(boundary, blocks, 0.1, diagonals_allowed = True)
print(boundary, blocks, start, goal)

# start = tuple((5,5.9,7.1))
# start = tuple((6.1,6.1,7))
# print(planner.cost(start, goal))

path, total_cost = planner.plan(start, goal)
points = np.array(path).T
print(f"The path is  given  by {path} and it costs {total_cost}")
print(f"The cost is {total_cost}")
fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points, figtitle = "A* on room", axtitle = "Generated Path" )
print(f"The number of  nodes sampled to reach = {len(planner.closed.keys())}")
print(f"The length of the path is {len(path)}")
print(f"The cost of reaching is given by")
print(f"The points sampled are")
points = np.array(list(planner.closed.keys())).T
fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points, fig = fig, axtitle = "Nodes Expanded")



In [ ]:
fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points, figtitle = "A* on room", axtitle = "Generated Path" )
print(f"The number of  nodes sampled to reach = {len(planner.closed.keys())}")
print(f"The length of the path is {len(path)}")
print(f"The cost of reaching is given by")
print(f"The points sampled are")
points = np.array(list(planner.closed.keys())).T
fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points, fig = fig, axtitle = "Nodes Expanded")

In [ ]:
planner = Djikstra(boundary, blocks, 0.3, diagonals_allowed = True)
# start = tuple((5,5.9,7.1))
# start = tuple((6.1,6.1,7))
print(boundary, blocks, start, goal)
print(planner.cost(start, goal))

path , total_cost= planner.plan(start, goal)
print(f"The path is  given  by {path} and it costs {total_cost}")
points = np.array(path).T

fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
print(f"The cost is {total_cost}")

print(f"The number of  nodes sampled to reach = {len(planner.closed.keys())}")
print(f"The length of the path is {len(path)}")

points = np.array(list(planner.closed.keys())).T
fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)

In [ ]:
with open("outputlog.txt",w) as f:
    

    for mapname in maps:
        if 'monza' in mapname:
            goal = np.array([2.3,12,4])
        elif 'flappy_bird' in mapname:
            goal = np.array([12,4,4])
        elif 'tower' in mapname:
            goal = np.array([4,4,12])
        elif 'room' in mapname:
            goal = np.array([8,8,2])
        
        f.write("-------------------------------------------------\n")
        # print("-------------------------------------------------")
        mapfile = os.path.join('./starter_code/maps/', mapname)
        boundary, blocks = load_map(mapfile)
        planner = A_star(boundary, blocks, 0.1, diagonals_allowed = True)
        f.write(f"The mapname is " +mapname)
        f.write(f"\nThe boundary is {boundary}")
        

        path, total_cost = planner.plan(start, goal)
        # if path is None:
        
        if path is not None:
            points = np.array(path).T
            f.write(f"\nThe path is  given  by {path}")
            f.write(f"\nThe cost is {total_cost}")
            fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
            
            f.write(f"\nThe number of  nodes sampled to reach = {len(planner.closed.keys())}")
            f.write(f"\nThe length of the path is {len(path)}")
            # print(f"The cost of reaching is given by")
            # f.write(f"\nThe points sampled are")
            points = np.array(list(planner.closed.keys())).T
            fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
        print("-------------------------------------------------")

In [ ]:
start = np.array([0.5, 0.5, 0.5])
goal = np.array([6.0, 6.0, 5.5])


    

for mapname in maps:
    if 'monza' in mapname:
        goal = np.array([2.3,12,4])
    elif 'flappy_bird' in mapname:
        goal = np.array([12,4,4])
    elif 'tower' in mapname:
        goal = np.array([4,4,12])
    elif 'room' in mapname:
        goal = np.array([8,8,2])
    print("-------------------------------------------------")
    # print("-------------------------------------------------")
    mapfile = os.path.join('./starter_code/maps/', mapname)
    boundary, blocks = load_map(mapfile)
    planner = A_star(boundary, blocks, 0.1, diagonals_allowed = True)
    print(f"The mapname is " +mapname)
    print(f"The boundary is {boundary}")
    

    path, total_cost = planner.plan(start, goal)
    # if path is None:
    
    if path is not None:
        points = np.array(path).T
        print(f"The path is  given  by {path}")
        print(f"The cost is {total_cost}")
        fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
        
        print(f"The number of  nodes sampled to reach = {len(planner.closed.keys())}")
        print(f"The length of the path is {len(path)}")
        # print(f"The cost of reaching is given by")
        print(f"The points sampled are")
        points = np.array(list(planner.closed.keys())).T
        fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
        # print("-------------------------------------------------")
    print("-------------------------------------------------")
    

In [ ]:
start = np.array([0.5, 0.5, 0.5])
goal = np.array([6.0, 6.0, 5.5])

for mapname in maps:
    if 'monza' in mapname:
        goal = np.array([2.3,12,4])
    elif 'flappy_bird' in mapname:
        goal = np.array([12,4,4])
    elif 'tower' in mapname:
        goal = np.array([4,4,12])
    elif 'room' in mapname:
        goal = np.array([8,8,2])
    print("-------------------------------------------------")
    # print("-------------------------------------------------")
    mapfile = os.path.join('./starter_code/maps/', mapname)
    boundary, blocks = load_map(mapfile)
    planner = Djikstra(boundary, blocks, 0.4, diagonals_allowed = True)
    print(f"The mapname is " +mapname)
    print(f"The boundary is {boundary}")
    

    path, total_cost = planner.plan(start, goal)
    # if path is None:
    
    if path is not None:
        points = np.array(path).T
        print(f"The path is  given  by {path}")
        print(f"The cost is {total_cost}")
        fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
        
        print(f"The number of  nodes sampled to reach = {len(planner.closed.keys())}")
        print(f"The length of the path is {len(path)}")
        # print(f"The cost of reaching is given by")
        print(f"The points sampled are")
        points = np.array(list(planner.closed.keys())).T
        fig, ax, hb, hs, hg = draw_map(boundary, blocks, start, goal,points)
        # print("-------------------------------------------------")
    print("-------------------------------------------------")
    

In [ ]:
from collections import defaultdict as dd
# a = tuple((1,2,3,))
# b = np.array(a)
# c = tuple(b)

# a = DotDict(default = None)
# b = tuple((0,0))
# a.b = 1
# a.new = 1
# a.old = 2
# print(a.values())
# # a.pop('new')
# a.keys(), a.values()
# # np.add(a,a)
# print(a.keys())
# a = {}
# b = 'New'
# a[b] = 1
# c = 5
# if not a[c]:
#     print(2)

a = dd(list)
if a[1] :
    print(1)
else:
    print(2)

from itertools import product

dir_list = (list(product([-1,0,1], repeat = 3)))
dir_list.remove((0,0,0))
for dir in dir_list:
    print(type(dir))
    break

dir_list

In [ ]:
a=np.array([1,2,3,4])
a[2:] = (5,6)
a